In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [5]:
def R5_test():
    print("Test-R5: Reward given out to block signers sums up to the total block reward")
    num = int(input("enter the number of blocks you want to test: "))
    
    current_block = getBlockNumber()
    next_block = current_block + 1
    print("current block", current_block, "next block", next_block)
    
    acc_rewards_prev = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            acc_rewards_prev[address] = reward_accumulated
            
    current_block = getBlockNumber()
    while current_block < next_block:
        current_block = getBlockNumber()
    
    iterations = 0
    while iterations < num:
        # get the validator info and compute validator rewards
        acc_rewards_curr = dict()
        validator_infos = getAllValidatorInformation()
        block_reward = 28e18
        validator_rewards = 0
        total_reward = 0
        signers = 0
        for i in validator_infos:
            if i['currently-in-committee'] == True:
                signers = signers + 1
                address = i['validator']['address']
                reward_accumulated = i['lifetime']['reward-accumulated']
                shard_metrics = i['metrics']['by-bls-key']
                validator_reward = 0
                for by_shard in shard_metrics:
                    validator_addr = by_shard['key']['earning-account']
                    by_shard_reward = block_reward * float(by_shard['key']['overall-percent']) / 0.32
                    validator_reward = validator_reward + by_shard_reward
                acc_rewards_curr[address] = reward_accumulated
                reward = reward_accumulated
                if address in acc_rewards_prev:
                    reward = reward - acc_rewards_prev[address]
                total_reward = total_reward + reward
                validator_rewards = validator_rewards + validator_reward
                
        print("Test-R5 pass?", (total_reward == validator_rewards), 
            "block", current_block, \
            "validator block reward", validator_rewards, \
            "total reward", "{:.20e}".format(total_reward), \
            "signers", signers)
        
        last_block = current_block
        current_block = getBlockNumber()
        while current_block == last_block:
            current_block = getBlockNumber()
        
        acc_rewards_prev = acc_rewards_curr
        
        iterations = iterations + 1
    print("done!!!")

In [6]:
R5_test()

Test-R5: Reward given out to block signers sums up to the total block reward
enter the number of blocks you want to test: 2
current block 2300 next block 2301
Test-R5 pass? False block 2301 validator block reward 5.5807944400095e+19 total reward 8.40000000000000000000e+19 signers 22
Test-R5 pass? False block 2302 validator block reward 5.5807944400095e+19 total reward 8.40000000000000000000e+19 signers 22
done!!!
